In [1]:
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
data = open("train.txt","r")
data=list(data)

test_data=open("test.txt","r")
test_data=list(test_data) 

In [0]:
def preprocess(data):
  tweets=[]
  sentiments=[]
  arr=[]
  c=0
  for i in data:
    if(i=="\n"):
      tweets.append(arr)
      arr=[]
    else:
      temp=i.split("\t")
      if(len(temp)==3):
        sentiments.append(temp[2])
      arr.append(temp[0])
  tweet=[]
  for i in tweets:
    temp=""
    for j in i:
      temp+=j
      temp+=" "
    tweet.append(temp[:-1])
  tweets=tweet 
  sent=[]
  for i in sentiments:
    temp=i[:-1]
    if(temp=="neutral"):
      sent.append(0)
    elif(temp=="positive"):
      sent.append(1)
    else:
      sent.append(2)  

  return(tweets,sent)

In [0]:
 tweets,sent=preprocess(data)

In [0]:
test_tweets,test_sent=preprocess(test_data)

In [0]:
df=pd.DataFrame()
df["tweet"]=tweets
test_df=pd.DataFrame()
test_df["tweet"]=test_tweets

In [0]:
import string

In [0]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["tweet"] = df["tweet"].apply(lambda text: remove_punctuation(text))
test_df["tweet"] = test_df["tweet"].apply(lambda text: remove_punctuation(text))

from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["tweet"] = df["tweet"].apply(lambda text: remove_stopwords(text))
test_df["tweet"] = test_df["tweet"].apply(lambda text: remove_stopwords(text))



#removing links
def removeLinks(df):
    for i in range(len(df['tweet'])):
        i_arr = df['tweet'][i].split(" ")
        if 'https' in i_arr:
            ind = i_arr.index('https')
            df['tweet'][i] = " ".join(i_arr[:ind])
    return df

df = removeLinks(df)
test_df = removeLinks(test_df)

In [0]:
tweets=df["tweet"].values
test_tweets=test_df["tweet"].values

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [0]:
import numpy as np
import pandas as pd




from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(tweets)

alphabet="abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    

tk.word_index = char_dict.copy() 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1



train_sequences = tk.texts_to_sequences(tweets)
test_texts = tk.texts_to_sequences(test_tweets)

train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')


train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')


train_classes = sent
train_class_list = [x for x in train_classes]

test_classes = test_sent
test_class_list = [x for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)


In [66]:
vocab_size = len(tk.word_index)
vocab_size

70

In [0]:
embedding_weights = [] #(71, 70)
embedding_weights.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [68]:
print(embedding_weights.shape) 
embedding_weights

(71, 70)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [0]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [0]:
input_size = 1014

embedding_size = 70
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 3
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [0]:
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [72]:
inputs = Input(shape=(input_size,), name='input', dtype='int64') 
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1014, 70)          4970      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1008, 256)         125696    
_________________________________________________________________
activation_13 (Activation)   (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 330, 256)          459008    
_________________________________________________________________
activation_14 (Activation)   (None, 330, 256)          0   

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_classes, 
                                                            random_state=1, test_size=0.15)

In [75]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=20,
          verbose=2)

Train on 12861 samples, validate on 2270 samples
Epoch 1/20
 - 21s - loss: 1.0727 - acc: 0.3961 - val_loss: 1.0126 - val_acc: 0.4555
Epoch 2/20
 - 21s - loss: 0.9846 - acc: 0.4923 - val_loss: 0.9639 - val_acc: 0.5137
Epoch 3/20
 - 20s - loss: 0.9001 - acc: 0.5514 - val_loss: 0.8989 - val_acc: 0.5696
Epoch 4/20
 - 20s - loss: 0.8315 - acc: 0.6035 - val_loss: 0.9189 - val_acc: 0.5705
Epoch 5/20
 - 20s - loss: 0.7616 - acc: 0.6485 - val_loss: 0.9497 - val_acc: 0.5595
Epoch 6/20
 - 21s - loss: 0.6619 - acc: 0.7078 - val_loss: 0.9180 - val_acc: 0.5837
Epoch 7/20
 - 20s - loss: 0.5598 - acc: 0.7666 - val_loss: 1.1175 - val_acc: 0.5665
Epoch 8/20
 - 20s - loss: 0.4659 - acc: 0.8145 - val_loss: 1.2272 - val_acc: 0.5718
Epoch 9/20
 - 21s - loss: 0.3989 - acc: 0.8439 - val_loss: 1.3325 - val_acc: 0.5740
Epoch 10/20
 - 20s - loss: 0.3003 - acc: 0.8926 - val_loss: 1.6888 - val_acc: 0.5357
Epoch 11/20
 - 20s - loss: 0.2525 - acc: 0.9104 - val_loss: 1.6552 - val_acc: 0.5537
Epoch 12/20
 - 20s - loss

In [0]:
pred_array=model.predict(test_data)

In [0]:
predicted=[]
for i in pred_array:
  predicted.append(np.argmax(i))

In [0]:
from sklearn.metrics import classification_report


In [0]:
test_result=[]
for i in test_classes:
  test_result.append(np.argmax(i))

In [0]:
import keras
y_pred=keras.utils.to_categorical(yp, num_classes=None)
y_true=keras.utils.to_categorical(test_classes, num_classes=None, dtype='float32')

In [80]:
target_names = ['neutral', 'positive', 'negative']
print(classification_report(test_result, predicted, target_names=target_names))

              precision    recall  f1-score   support

     neutral       0.47      0.60      0.52       754
    positive       0.52      0.48      0.50       582
    negative       0.57      0.40      0.47       533

    accuracy                           0.50      1869
   macro avg       0.52      0.49      0.50      1869
weighted avg       0.51      0.50      0.50      1869

